In [ ]:
!pip install lightfm
!pip install implicit==0.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 10.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705373 sha256=ec9c4ba7476f265ee19b0adf92a8122466bf0344834991078db5e8e8fbbe857b
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 13.5 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3423209 sha256=454a30b8fb73a96b3d0a33fccf215198dda47c091920968f2f6a550a097ae794
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

# Функции из 1-ого вебинара
import os, sys

import itertools

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  PATH = '/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_3/'

else:

  PATH = 'D:/'

Mounted at /content/drive


In [ ]:
try:

  from metrics import precision_at_k as custom_precision, recall_at_k
  from utils import prefilter_items
  # from recommenders import MainRecommender

except:
  
  sys.path.insert(0,'/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_5/')
  from metrics import precision_at_k as custom_precision, recall_at_k
  from utils import prefilter_items
  # from recommenders import MainRecommender

from lightfm.evaluation import precision_at_k, recall_at_k
# from metrics import precision_at_k as custom_precision, recall_at_k
# from utils import prefilter_items

In [ ]:
try:
  
  data = pd.read_csv('../data/retail_train.csv')
  item_features = pd.read_csv('../data/product.csv')
  user_features = pd.read_csv('../data/hh_demographic.csv')

except:

  data = pd.read_csv(PATH + 'data/retail_train.csv')
  item_features = pd.read_csv(PATH + '/data/product.csv') 
  user_features = pd.read_csv(PATH + '/data/hh_demographic.csv') 

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [ ]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [ ]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [ ]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

## 1. Filter items

In [ ]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

/content/drive/MyDrive/Move_to/GEEK_brain/Рекомендательные системы/webinar_5/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [ ]:

user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [ ]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [ ]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
user_feat.shape

(2497, 7)

In [ ]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
117847,450.0,NUTRITION,National,REFRIGERATED,SOY/RICE MILK,64 OZ
818981,194.0,GROCERY,National,COLD CEREAL,ALL FAMILY CEREAL,10.4 OZ


In [ ]:
item_feat.shape

(5001, 6)

## Encoding features

In [ ]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [ ]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Домашнее задание

#### 1) Прочитать статьи про BPR, WARP loss

#### 2) Сделать грид серч текущей модели, смотрите на метрику precision@5, считаем на тесте нашей функцией
    Подбор параметров:
    - Loss
    - no_components
    - regularization

In [ ]:
def greed_search_my(**params):

  model = LightFM(**params)
  
  model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 
  
  return model

In [ ]:
def get_df_result_for_metrics(data_users_ids_row, data_items_ids_row, my_model):

  # подготавливаемм id для юзеров и товаров в порядке пар user-item
  users_ids_row = data_users_ids_row.apply(lambda x: userid_to_id[x]).values.astype(int)
  items_ids_row = data_items_ids_row.apply(lambda x: itemid_to_id[x]).values.astype(int)

  # модель возвращает меру/скор похожести между соответствующим пользователем и товаром
  predictions = my_model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)
  
  data_train_filtered['score'] = predictions

  # создаем предикт датафрейм в формате списка това
  predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
              unique().reset_index()

  # объединяем предикт и тест датасет для подсчета precision
  return result.merge(predict_result, on='user_id', how='inner')     



In [ ]:
%%time

lst_loss = []
lst_no_components = []
lst_precision = []
lst_learning_rate = []

best_precision = 0

params = {

    'loss' : ['bpr',  "warp"],
    'no_components': [32, 64, 128, 256],
    'learning_rate': [0.01, 0.05, 0.1], 
    'item_alpha': [0.4],
    'user_alpha': [0.1], 
    'random_state': [42],
    'k': [5],
    'n': [15],
    'max_sampled': [100]
    
}

keys, values = zip(*params.items())
params_all = [dict(zip(keys, v)) for v in itertools.product(*values)]

best_score = 0

for num_now in range(len(params_all)):

  model = greed_search_my(**params_all[num_now]) 
  df_result_for_metrics = get_df_result_for_metrics(data_train_filtered['user_id'], data_train_filtered['item_id'], my_model = model)
  precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()

  lst_loss.append(params_all[num_now]['loss'])
  lst_no_components.append(params_all[num_now]['no_components'])
  lst_learning_rate.append(params_all[num_now]['learning_rate'])
  lst_precision.append(precision)
 

  if precision > best_precision:
      best_precision = precision
      best_model = model

Epoch: 100%|██████████| 20/20 [05:08<00:00, 15.42s/it]


CPU times: user 1h 49min 33s, sys: 5.09 s, total: 1h 49min 38s
Wall time: 57min 7s


In [ ]:
for_plot = pd.DataFrame([lst_loss, lst_no_components, lst_precision, lst_learning_rate], index = ['loss', 'comp', 'precision', 'learning_rate']).T
for_plot

,loss,comp,precision,learning_rate
0,bpr,32,0.145156,0.01
1,bpr,32,0.134319,0.05
2,bpr,32,0.135107,0.1
3,bpr,64,0.141511,0.01
4,bpr,64,0.05619,0.05
5,bpr,64,0.061018,0.1
6,bpr,128,0.143383,0.01
7,bpr,128,0.062791,0.05
8,bpr,128,0.053432,0.1
9,bpr,256,0.142496,0.01


In [ ]:
import plotly.express as px
    

# columns=['factors', 'regularization', 'precision_at_k']    
# xyz = pd.DataFrame([[i.split('_')[0] for i in list(metric_value.keys())], [i.split('_')[1] for i in list(metric_value.keys())], metric_value.values()], index = columns).T

fig = px.scatter_3d(for_plot, 
                    x='comp', 
                    y='learning_rate', 
                    z='precision', 
                    color='loss',
                    title= 'Зависимость precision_at_k от ',
                    width=1000,
                    height=1000)


fig.show()

# Links

Neural networks for RS: http://d2l.ai/chapter_recommender-systems/mf.html

LigthFM -> https://arxiv.org/pdf/1507.08439.pdf

https://making.lyst.com/lightfm/docs/home.html
